In [1]:
import pandas as pd
import numpy as np

# Aggregating all previous data

## Notes

Taking another look at the data prior to writing the software. This also defines the methods of collecting the data from the app-dev standpoint. For the moment we can use the contents of <data/from_prev/u_all_data.csv> as a proposed data model.

### quantity

In [2]:
a = pd.read_csv("data/from_prev/agg_results_with_land_use_2015.csv")
not_these = [
    'sihlsee_einsiedeln_schilligerllacherl',
    'schiffenensee_duedingen_hirschij',
    'lac-leman-hammerdirt'
]
    
a = a[~a.location.isin(not_these)].copy()
a_cols = [
    'loc_date',
    'date',
    'water_name_slug',
    'location',
    'code',
    'pcs_m',
    'quantity',
    'river_bassin',
    'length', 
    'groupname',
    'city']

a_c = a[a_cols].copy()

In [3]:
def collect_vitals(data):
    total = data.quantity.sum()
    median = data.pcs_m.median()
    samples = data.loc_date.nunique()
    ncodes = data.code.nunique()
    nlocations = data.location.nunique()
    nbodies = data.water_name_slug.nunique()
    return total, median, samples, ncodes, nlocations, nbodies

b = pd.read_csv("data/from_prev/u_all_data.csv")
b = b[~b.loc_date.isin(not_these)].copy()
b_vitals = collect_vitals(b)
b_vitals

(192380, 0.0, 1432, 228, 232, 54)

In [4]:
a_c_vitals = collect_vitals(a_c)
a_c_vitals

(198824, 0.0, 1449, 184, 252, 61)

In [5]:
def find_missing(more_than, less_than):
    return np.setdiff1d(more_than, less_than)

missing = find_missing(a_c.loc_date.unique(), b.loc_date.unique())
matched_a_b = a_c[~a_c.loc_date.isin(missing)].copy()
diff_a_b = a_c[a_c.loc_date.isin(missing)]

In [6]:
matched_vitals = collect_vitals(matched_a_b)
matched_vitals

(191048, 0.0, 1429, 184, 232, 54)

In [7]:
missing_m = find_missing(b.loc_date.unique(), matched_a_b.loc_date.unique())
missing_matched = b[b.loc_date.isin(missing_m)].copy()
matched_b_vitals = collect_vitals(missing_matched)

In [8]:
matched_vitals[0] + matched_b_vitals[0] - b_vitals[0]

0

### codes

In [9]:
dfCodes = pd.read_csv("data/end_process/codes.csv")
b_all_match = b[~b.loc_date.isin(missing_m)]
b_a_m_vitals = collect_vitals(b_all_match)
a_codes = matched_a_b.code.unique()
b_codes = b_all_match.code.unique()

# these are codes from the alpes and other codes established after 2020
# these codes are in b and not in a
not_in_a = find_missing(b_codes, a_codes)

In [10]:
# these codes are in a not in b
# these are the aggregated codes for Gfoam, Gfrags and Gcaps
not_in_b = find_missing(a_codes, b_codes)

In [11]:
b_a_m_vitals

(191048, 0.0, 1429, 176, 232, 54)

In [12]:
matched_vitals

(191048, 0.0, 1429, 184, 232, 54)

In [13]:
columns = ['code', 'material', 'description', 'source', 'parent_code', 'single_use', 'groupname']

In [14]:
dfCodes = dfCodes[columns].copy()
dfCodes = dfCodes.set_index("code", drop=True)

In [15]:
not_in_b

array(['G21', 'G23', 'G24', 'G75', 'G76', 'G78', 'G79', 'G80', 'G81',
       'G82', 'G83'], dtype=object)

In [16]:
not_in_a

array(['Gcaps', 'Gfoam', 'Gfrags'], dtype=object)

In [17]:
dfCodes.head()

,material,description,source,parent_code,single_use,groupname
code,,,,,,
G708,Metal,Batons de ski,Usagers,G199,False,recreation
G212,Chemicals,Oil nodules or coal fragments -- not from a BBQ,Undefined,Parent code,False,unclassified
G213,Chemicals,Paraffin wax,Undefined,Parent code,False,recreation
G214,Chemicals,Oil/tar,Construction,Parent code,False,infrastructure
G135,Cloth,"Clothes, footware, headware, gloves",Clothing,Parent code,False,personal items


In [18]:
# dfCodes.loc[['G21', 'G22','G23', 'G24'], "parent_code"] = "Gcaps"

# dfCodes.loc[['G75', 'G78', 'G79',"G80"], "parent_code"] = "Gfrags"
# dfCodes.loc[['G76','G81','G82', 'G83'], "parent_code"] = "Gfoams"

def aggregate_gcaps_gfoams_gfrags(data, codes,columns=["Gfoams", "Gfrags", "Gcaps"]):
    for col in columns:
        change = codes.loc[codes.parent_code == col].index
        data.loc[data.code.isin(change), "code"] = col
        
    return data

matched_a_b_aggregated = aggregate_gcaps_gfoams_gfrags(matched_a_b.copy(), dfCodes)

In [19]:
code_cols = ['material', 'description', 'source', 'parent_code', 'single_use', 'groupname']



In [20]:
matched_a_b.loc[matched_a_b.code.isin(['G75', 'G78', 'G79',"G80"])]

,loc_date,date,water_name_slug,location,code,pcs_m,quantity,river_bassin,length,groupname,city
9,"('aabach', '2020-10-22')",2020-10-22,zurichsee,aabach,G78,0.20,6,linth,30.000000,plastic pieces,Schmerikon
10,"('aabach', '2020-10-22')",2020-10-22,zurichsee,aabach,G79,0.03,1,linth,30.000000,plastic pieces,Schmerikon
28,"('aare-limmatspitz', '2020-07-13')",2020-07-13,aare,aare-limmatspitz,G78,0.04,5,aare,120.000000,plastic pieces,Gebenstorf
29,"('aare-limmatspitz', '2020-07-13')",2020-07-13,aare,aare-limmatspitz,G79,0.03,4,aare,120.000000,plastic pieces,Gebenstorf
52,"('aare-port', '2021-04-23')",2021-04-23,aarenidau-buren-kanal,aare-port,G78,0.12,5,aare,40.000000,plastic pieces,Port
...,...,...,...,...,...,...,...,...,...,...,...
268380,"('jardin-botanique', '2016-04-24')",2016-04-24,lac-leman,jardin-botanique,G80,0.00,0,rhone,13.019391,plastic pieces,Genève
268404,"('jardin-botanique', '2016-04-24')",2016-04-24,lac-leman,jardin-botanique,G75,0.00,0,rhone,13.019391,plastic pieces,Genève
268485,"('le-port', '2016-04-19')",2016-04-19,lac-leman,le-port,G78,0.00,0,rhone,13.019391,plastic pieces,La Tour-de-Peilz
268526,"('le-port', '2016-04-19')",2016-04-19,lac-leman,le-port,G80,0.00,0,rhone,13.019391,plastic pieces,La Tour-de-Peilz


In [21]:
collect_vitals(matched_a_b_aggregated)

(191048, 0.0, 1429, 175, 232, 54)

In [22]:
collect_vitals(matched_a_b)

(191048, 0.0, 1429, 184, 232, 54)

In [23]:
collect_vitals(b_all_match)

(191048, 0.0, 1429, 176, 232, 54)

In [24]:
cone = b_all_match.code.unique()
ctwo = matched_a_b_aggregated.code.unique()

find_missing(cone, ctwo)

array(['G22', 'Gfoam'], dtype=object)

In [25]:
matched_a_b_aggregated[matched_a_b_aggregated.code.isin(['G75', 'G78', 'G79',"G80"])]

,loc_date,date,water_name_slug,location,code,pcs_m,quantity,river_bassin,length,groupname,city


### survey areas

In [26]:
collect_vitals(diff_a_b)

(7776, 0.0, 20, 184, 20, 7)

In [27]:
f = pd.read_csv("data/from_prev/checked_alpes_survey_data.csv")
collect_vitals(f)

(7776, 0.0, 20, 230, 20, 7)

In [28]:
cmore = f.code.unique()
cless = diff_a_b.code.unique()

find_missing(cmore, cless)

array(['G702', 'G703', 'G704', 'G705', 'G706', 'G707', 'G708', 'G709',
       'G710', 'G711', 'G712', 'G713', 'G900', 'G901', 'G902', 'G903',
       'G904', 'G905', 'G906', 'G907', 'G908', 'G913', 'G914', 'G915',
       'G916', 'G917', 'G918', 'G919', 'G921', 'G922', 'G923', 'G925',
       'G926', 'G927', 'G928', 'G929', 'G930', 'G931', 'G932', 'G933',
       'G934', 'G935', 'G936', 'G937', 'G938', 'G939', 'G940', 'G941',
       'G942', 'G943', 'G944', 'G945', 'Gfoam', 'Gfrags'], dtype=object)

In [29]:
f = f[a_cols].copy()
f["loc_date"] = list(zip(f.location, f.date))
f.head()

,loc_date,date,water_name_slug,location,code,pcs_m,quantity,river_bassin,length,groupname,city
0,"(cabanes-des-diablerets, 2021-08-28)",2021-08-28,alpes-vaudoises,cabanes-des-diablerets,G200,1.33,16,les-alpes,12.0,food and drink,Ormont-Dessus
1,"(cabanes-des-diablerets, 2021-08-28)",2021-08-28,alpes-vaudoises,cabanes-des-diablerets,G48,0.42,5,les-alpes,12.0,recreation,Ormont-Dessus
2,"(cabanes-des-diablerets, 2021-08-28)",2021-08-28,alpes-vaudoises,cabanes-des-diablerets,G170,0.83,10,les-alpes,12.0,agriculture,Ormont-Dessus
3,"(cabanes-des-diablerets, 2021-08-28)",2021-08-28,alpes-vaudoises,cabanes-des-diablerets,G27,0.75,9,les-alpes,12.0,tobacco,Ormont-Dessus
4,"(cabanes-des-diablerets, 2021-08-28)",2021-08-28,alpes-vaudoises,cabanes-des-diablerets,G709,0.17,2,les-alpes,12.0,recreation,Ormont-Dessus


In [30]:
matched_a_b[a_cols].head()

,loc_date,date,water_name_slug,location,code,pcs_m,quantity,river_bassin,length,groupname,city
0,"('aabach', '2020-10-22')",2020-10-22,zurichsee,aabach,G112,0.23,7,linth,30.0,micro plastics (< 5mm),Schmerikon
1,"('aabach', '2020-10-22')",2020-10-22,zurichsee,aabach,G124,0.06,2,linth,30.0,unclassified,Schmerikon
2,"('aabach', '2020-10-22')",2020-10-22,zurichsee,aabach,G23,0.03,1,linth,30.0,packaging non food,Schmerikon
3,"('aabach', '2020-10-22')",2020-10-22,zurichsee,aabach,G27,0.23,7,linth,30.0,tobacco,Schmerikon
4,"('aabach', '2020-10-22')",2020-10-22,zurichsee,aabach,G30,0.10,3,linth,30.0,food and drink,Schmerikon


In [31]:
merge_cols = a_cols.copy()

In [32]:
alr = alpes_and_lakes_and_rivers = pd.concat([f[merge_cols], matched_a_b[merge_cols]])

In [33]:
alr['date'] = pd.to_datetime(alr["date"])

In [34]:
alr['date'].max()

Timestamp('2021-08-28 00:00:00')

In [35]:
alr["river_bassin"].unique()

array(['les-alpes', 'linth', 'aare', 'rhone', 'ticino', 'rhine'],
      dtype=object)

In [36]:
alr.head()

,loc_date,date,water_name_slug,location,code,pcs_m,quantity,river_bassin,length,groupname,city
0,"(cabanes-des-diablerets, 2021-08-28)",2021-08-28,alpes-vaudoises,cabanes-des-diablerets,G200,1.33,16,les-alpes,12.0,food and drink,Ormont-Dessus
1,"(cabanes-des-diablerets, 2021-08-28)",2021-08-28,alpes-vaudoises,cabanes-des-diablerets,G48,0.42,5,les-alpes,12.0,recreation,Ormont-Dessus
2,"(cabanes-des-diablerets, 2021-08-28)",2021-08-28,alpes-vaudoises,cabanes-des-diablerets,G170,0.83,10,les-alpes,12.0,agriculture,Ormont-Dessus
3,"(cabanes-des-diablerets, 2021-08-28)",2021-08-28,alpes-vaudoises,cabanes-des-diablerets,G27,0.75,9,les-alpes,12.0,tobacco,Ormont-Dessus
4,"(cabanes-des-diablerets, 2021-08-28)",2021-08-28,alpes-vaudoises,cabanes-des-diablerets,G709,0.17,2,les-alpes,12.0,recreation,Ormont-Dessus


In [37]:
# alr = alr.fillna("hello")
alr.isna().any()

loc_date           False
date               False
water_name_slug    False
location           False
code               False
pcs_m              False
quantity           False
river_bassin       False
length             False
groupname          False
city               False
dtype: bool

### after august 2021

In [38]:

# dfCodes.loc[["Gfrags", "Gfoam"]]

# gcaps = {"material":"plastic", "description":"plastic caps, lid rings: G21, G22, G23, G24", "source":'Usagers',
#          "parent_code":"Parent code", "single_use":"False", "groupname": "packaging non food"}

# dfCodes.loc["Gcaps", :] = gcaps
# dfCodes.loc["Gfoams", :] = dfCodes.loc["Gfoam"]
# dfCodes.drop("Gfoam", inplace=True)

In [39]:
dfCodes.loc["Gcaps"]

material                                           plastic
description    plastic caps, lid rings: G21, G22, G23, G24
source                                             Usagers
parent_code                                    Parent code
single_use                                           False
groupname                               packaging non food
Name: Gcaps, dtype: object

In [40]:
# dfCodes[code_cols].to_csv("data/end_process/codes.csv", index=True)

In [44]:
f = pd.read_csv("data/from_prev/u_sup_after.csv")
f.rename(columns={"pcs/m":"pcs_m"}, inplace=True)

f["length"] = (f.quantity/f.pcs_m)
collect_vitals(f)

(2938, 0.0, 6, 228, 5, 1)

In [46]:
fill_in_length = f[["loc_date","length"]].drop_duplicates(["loc_date", "length"]).dropna()
# fill_in_length = fill_in_length.groupby("loc_date").length.mean().astype("int")
fill_in_length

,loc_date,length
1,"('maladaire', '2021-06-02')",63.636364
2,"('maladaire', '2021-06-02')",66.666667
25,"('maladaire', '2021-06-02')",53.846154
34,"('maladaire', '2021-06-02')",50.000000
60,"('maladaire', '2021-06-02')",60.000000
76,"('maladaire', '2021-06-02')",62.500000
77,"('maladaire', '2021-06-02')",57.142857
98,"('maladaire', '2021-06-02')",61.538462
113,"('maladaire', '2021-06-02')",60.824742
152,"('maladaire', '2021-06-02')",61.111111


In [ ]:
f['length'] =f.loc_date.apply(lambda x: fill_in_length.loc[x])
f.loc[f.code == "Gfoam",  "code"] = "Gfoams"

In [ ]:
f["groupname"] = f.code.apply(lambda x: dfCodes["groupname"].loc[x])

In [ ]:
f.isna().any()

In [ ]:
# alr_pm = alr_plus_missing = pd.concat([alr[merge_cols], missing_matched[merge_cols]])

In [ ]:
alr_pm.head()

In [ ]:
collect_vitals(missing_matched)

In [ ]:
missing_matched.loc_date.unique()

In [ ]:
f.loc_date.unique()